In [1]:
!nvidia-smi

Wed Jan 10 05:31:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Import Libraries

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

## rouge_score--> Performance Metric for Text Summarization

In [3]:
!pip install --upgrade accelerate
!pip uminstall -y transformers accelerate
!pip install transformers accelerate

ERROR: unknown command "uminstall" - maybe you meant "uninstall"


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from transformers import AutoModelForSeq2SeqLM,   AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from tqdm import tqdm
import torch

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' ## To check GPU is connected or not
device

'cuda'

# Load Tokenizer and Model
## Model URL: https://huggingface.co/google/pegasus-cnn_dailymail

In [6]:
from transformers import AutoModelForSeq2SeqLM,   AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"  ## Model Check-Point

tokenizer = AutoTokenizer.from_pretrained(model_ckpt) ## Tokenizer for above model

### Everry model has their own Tokenizer

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device) ## Load Model in device(here cuda) Memory




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load samsum Dataset form Hugging-Face
###  Dataset URL: https://huggingface.co/datasets/samsum

In [8]:
from datasets import load_dataset, load_from_disk, load_metric

dataset_samsum = load_dataset('samsum')


In [9]:
### Meta Data of Dataset in DatasetDict [to prevent memory load issue]
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [10]:
dataset_samsum['train'].column_names

['id', 'dialogue', 'summary']

In [11]:
dataset_samsum['train'].features

{'id': Value(dtype='string', id=None),
 'dialogue': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [12]:
len(dataset_samsum['train'].features)

3

In [13]:
[len((dataset_samsum[split])) for split in dataset_samsum]

[14732, 819, 818]

In [14]:
# dataset_samsum['train']['dialogue']

In [15]:
# dataset_samsum['train']['summary']

In [16]:
type(dataset_samsum)

datasets.dataset_dict.DatasetDict

In [17]:
type(dataset_samsum['train'])

datasets.arrow_dataset.Dataset

In [18]:
type(dataset_samsum['train']['dialogue'])

list

## Dataset Description and Split

In [19]:
split_length = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Train_Split: {split_length[0]}\nTest_split: {split_length[1]}\nValidation_split: {split_length[2]}")

print(f"No. of Features: {len(dataset_samsum['train'].features)}\nFeatures Names: {dataset_samsum['train'].column_names}")


Train_Split: 14732
Test_split: 819
Validation_split: 818
No. of Features: 3
Features Names: ['id', 'dialogue', 'summary']


## Tokenization, Mask Embeddings

In [20]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

  return {
      'input_ids' : input_encodings['input_ids'],
      'attention_mask' : input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }



In [21]:

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [22]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [23]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [24]:
dataset_samsum_pt['train']['dialogue'][0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [25]:
dataset_samsum_pt['train']['input_ids'][0]

[12195,
 151,
 125,
 7091,
 3659,
 107,
 842,
 119,
 245,
 181,
 152,
 10508,
 151,
 7435,
 147,
 12195,
 151,
 125,
 131,
 267,
 650,
 119,
 3469,
 29344,
 1]

In [26]:
dataset_samsum_pt['train']['attention_mask'][0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [27]:
dataset_samsum_pt['train']['labels'][0]

[12195, 7091, 3659, 111, 138, 650, 10508, 181, 3469, 107, 1]

## Training

In [28]:
from transformers import DataCollatorForSeq2Seq

### To load dataste as btches in memory

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

# Training Arguement

In [29]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir = 'pegasus-samsum',
    num_train_epochs = 1,
    warmup_steps = 500,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)


# Training Object for Training Model


In [30]:
trainer = Trainer(
    model = model_pegasus,
    args = trainer_args,
    tokenizer = tokenizer,
    data_collator = seq2seq_data_collator,
    train_dataset = dataset_samsum_pt['test'], ## Change to train *****
    eval_dataset = dataset_samsum_pt['validation']
)


## Call Trainer Object for Training

In [31]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 146.1132, 'train_samples_per_second': 5.605, 'train_steps_per_second': 0.349, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 1.0})

## Evaluation of Model

In [32]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [33]:
rouge_names = ['rouge1','rouge2', 'rougeL', 'rougeLsum']
rouge_metric = load_metric('rouge')
rouge_metric

<ipython-input-33-2756a93ebae1>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [39]:
##  While actual training set below test[0:10] as test only for complete test data.  ********

score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )


100%|██████████| 5/5 [00:14<00:00,  2.90s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018989,0.0,0.018895,0.018803


# Save Model

In [35]:
model_pegasus.save_pretrained('pegasus-samsum-mdoel')

## Save Tokenizer

In [36]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

## Load Tokenizer

In [37]:
# Load

tokenizer = AutoTokenizer.from_pretrained('/content/tokenizer')


# Prediction

In [40]:
#Prediction
from transformers import pipeline

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 130}  ## max_length: no. of chars in summary



sample_text = dataset_samsum["test"][4]["dialogue"]

reference = dataset_samsum["test"][4]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-mdoel",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Dialogue:
Ollie: Hi , are you in Warsaw
Jane: yes, just back! Btw are you free for diner the 19th?
Ollie: nope!
Jane: and the  18th?
Ollie: nope, we have this party and you must be there, remember?
Jane: oh right! i lost my calendar..  thanks for reminding me
Ollie: we have lunch this week?
Jane: with pleasure!
Ollie: friday?
Jane: ok
Jane: what do you mean " we don't have any more whisky!" lol..
Ollie: what!!!
Jane: you just call me and the all thing i heard was that sentence about whisky... what's wrong with you?
Ollie: oh oh... very strange! i have to be carefull may be there is some spy in my mobile! lol
Jane: dont' worry, we'll check on friday.
Ollie: don't forget to bring some sun with you
Jane: I can't wait to be in Morocco..
Ollie: enjoy and see you friday
Jane: sorry Ollie, i'm very busy, i won't have time for lunch  tomorrow, but may be at 6pm after my courses?this trip to Morocco was so nice, but time consuming!
Ollie: ok for tea!
Jane: I'm on my way..
Ollie: tea is ready, d